In [1]:
import cv2
import sys
import os
import time
import dlib
import random
import numpy as np
import pandas as pd
from skimage import io
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from sklearn.decomposition import PCA
from sklearn.cross_validation  import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

C:\Users\test\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\test\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [11]:
#使用dlib库进行人脸检测
cascPath = "haarcascade_frontalface_default.xml"
dataPath='D:\\code\\pycode\\faceRecognition\\database\\'
patchPath='D:\\code\\pycode\\faceRecognition\\faceData\\'
faceCascade = cv2.CascadeClassifier(cascPath)
for personFolder in os.listdir(dataPath):
    personPath=os.path.join(dataPath,personFolder)
    for fileName in os.listdir(personPath):
        #遍历文件夹下所有图片
        image = cv2.imread(os.path.join(personPath,fileName))
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        #将图片转为灰度模式
        faces = faceCascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(30, 30))
        if(len(faces)==1):
            x,y,w,h=list(faces[0])
            facePatch=image[y:y+h,x:x+w]
            if(not os.path.exists(os.path.join(patchPath,personFolder))):
                os.mkdir(os.path.join(patchPath,personFolder)) 
            #保存所有检测到的人脸
            cv2.imwrite(os.path.join(patchPath+personFolder,fileName),facePatch)

In [ ]:
#使用ImageDataGenerator进行数据增强
detector = dlib.get_frontal_face_detector()
dataPath='D:\\code\\pycode\\faceRecognition\\database\\'
patchPath='D:\\code\\pycode\\faceRecognition\\faceData1\\'
detector = dlib.get_frontal_face_detector()
datagen = ImageDataGenerator(
        rotation_range=20,
#        width_shift_range=0.2,
#        height_shift_range=0.2,
        rescale=None,
        shear_range=0.1,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        data_format='channels_last',
        cval=0,
        #channel_shift_range=0,
        vertical_flip=False)
for personFolder in os.listdir(dataPath):
    personPath=os.path.join(dataPath,personFolder)
    for fileName in os.listdir(personPath):
        image = cv2.imread(os.path.join(personPath,fileName))
        dets = detector(image, 1)
        for i, d in enumerate(dets):
            print("person{} face{} Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(personFolder,fileName[-5:-4],i, d.left(), d.top(), d.right(), d.bottom()))
            facePatch=image[d.top():d.bottom(),d.left():d.right()]
            cv2.imwrite(os.path.join(patchPath+personFolder,fileName),facePatch)
            
            facePatch=img_to_array(facePatch)
            facePatch = facePatch.reshape((1,) + facePatch.shape)
            if(not os.path.exists(os.path.join(patchPath,personFolder))):
                os.mkdir(os.path.join(patchPath,personFolder)) 
                
            i = 0
            for batch in datagen.flow(facePatch, batch_size=5,
                                      save_to_dir=None#os.path.join(patchPath,personFolder), 
                                      #save_prefix=personFolder, 
                                      #save_format='jpeg'
                                      ):
                cv2.imwrite(os.path.join(patchPath+personFolder,fileName[:-4]+str(i)+'.JPG'),batch[0].astype(int))
                i += 1
                if i > 5:
                    break  # 否则生成器会退出循环
            


In [4]:
#读取所有人脸数据，并使用PCA进行降维
trainData=[[]]
label=[]
dataPath='D:\\code\\pycode\\faceRecognition\\faceData1\\'
for dataLabel in os.listdir(dataPath):
    folderPath=os.path.join(dataPath,dataLabel)
    for fileName in os.listdir(folderPath):
        image=cv2.imread(os.path.join(folderPath,fileName))
        gray_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        imageArray=np.array(cv2.resize(gray_image,(101,101))).reshape([101*101])/255
        label.append(int(dataLabel))
        if(trainData==[[]]):
            trainData = [imageArray]
            continue
        trainData = np.concatenate((trainData, [imageArray]),axis=0)
pca = PCA(n_components=0.95)
pca.fit(trainData)
#print ("explained_variance_ratio_",pca.explained_variance_ratio_)
#print ("explained_variance_",pca.explained_variance_)
print ("pca.n_components_",pca.n_components_)
label=np.array(label)
trainData=pca.transform(trainData)
    train_images, test_images, train_labels, test_labels = train_test_split(trainData, label, test_size = 0.1, random_state = random.randint(0, 100)) 

C:\Users\test\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  # This is added back by InteractiveShellApp.init_path()


explained_variance_ [9.30563394e+01 2.59330455e+01 1.41939514e+01 8.40064569e+00
 7.82357035e+00 6.42211011e+00 6.13298868e+00 4.03662275e+00
 3.64808265e+00 2.95069853e+00 2.72460899e+00 2.18832621e+00
 2.07116385e+00 1.88104333e+00 1.71905881e+00 1.66628237e+00
 1.39929170e+00 1.25982322e+00 1.14435455e+00 1.07452002e+00
 1.02019881e+00 9.84561914e-01 9.21918097e-01 8.95024320e-01
 8.41218152e-01 7.96008563e-01 7.83868748e-01 7.36044410e-01
 7.10644629e-01 6.59112705e-01 6.50906526e-01 6.01274336e-01
 5.80219340e-01 5.63628472e-01 5.34013969e-01 5.15436771e-01
 4.84377142e-01 4.70503563e-01 4.55146570e-01 4.42822087e-01
 4.14670610e-01 4.00608457e-01 3.95222584e-01 3.87632994e-01
 3.74686138e-01 3.64963239e-01 3.54545247e-01 3.46966623e-01
 3.34975085e-01 3.32642584e-01 3.09766357e-01 3.07493404e-01
 3.03037957e-01 2.94842128e-01 2.89603262e-01 2.87722256e-01
 2.78269778e-01 2.70458588e-01 2.65288739e-01 2.61039397e-01
 2.49694320e-01 2.40479758e-01 2.37044316e-01 2.33929066e-01
 2.2

In [5]:
#使用SVM进行分类
svc = SVC(gamma='auto')
svc.fit(train_images,train_labels)   #训练模型
test_pred=svc.predict(test_images)   #用测试数据来做预测
print(classification_report(test_labels, test_pred))

             precision    recall  f1-score   support

          1       0.50      1.00      0.67         1
          2       1.00      0.30      0.46        10
          3       1.00      0.17      0.29         6
          4       1.00      0.25      0.40         4
          5       0.00      0.00      0.00         2
          6       0.33      0.20      0.25         5
          7       0.14      0.50      0.22         2
          8       0.33      0.20      0.25         5
          9       0.33      0.50      0.40         2
         10       0.00      0.00      0.00         3
         11       0.50      0.17      0.25         6
         12       0.33      1.00      0.50         2
         13       0.67      0.40      0.50         5
         14       0.00      0.00      0.00         4
         15       0.00      0.00      0.00         1
         16       0.00      0.00      0.00         1
         17       0.00      0.00      0.00         1
         18       1.00      0.50      0.67   

C:\Users\test\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\test\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [ ]:
clf = RandomForestClassifier(n_jobs=111)
clf.fit(train_images,train_labels) 
test_pred=clf.predict(test_images)  #用测试数据来做预测
print(classification_report(test_labels,test_pred))

In [ ]:
clf = AdaBoostClassifier(n_estimators=100)
